In [26]:
import pandas as pd
import numpy as np
from ETL_HOTELES.src import soporte_limpieza as sl
pd.set_option('display.max_columns', None)

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


ModuleNotFoundError: No module named 'ETL_HOTELES'

# Fases:

- Carga Dataset
- Inspección
- Nulos
- Outliers

### CARGA DATASET Y VISUALIZACIÓN

In [4]:
df_raw = pd.read_parquet("../data/raw/reservas_hoteles.parquet", engine = 'auto')

### Generamos una copia para trabajar sobre ella y mantrener intacto el archivo original

In [5]:
df = df_raw.copy()

### Eliminamos duplicados exactos

In [6]:
df.drop_duplicates(inplace=True)

### Asignamos tipo de dato correctos a las columnas de fecha

In [8]:
columnas_a_datetime = ["fecha_reserva", "inicio_estancia", "final_estancia"]
df = convertir_columnas_datetime(df, columnas_a_datetime)

Advertencia: En la columna 'fecha_reserva' se han asignado NaT en 5172 de 15000 registros (34.48 %).
Advertencia: En la columna 'inicio_estancia' se han asignado NaT en 75 de 15000 registros (0.5 %).
Advertencia: En la columna 'final_estancia' se han asignado NaT en 75 de 15000 registros (0.5 %).


**2. Asegurar la unicidad del correo electrónico por cliente**  
  

    La forma más eficiente de reasignar los id_cliente sería asociándolo al mail, que debería ser único

In [9]:
df['id_cliente'] = df.groupby('mail').ngroup() + 1

SEPARAMOS LOS HOTELES PROPIOS Y DE LA COMPETENCIA EN DATAFRAMES DISTINTOS PARA TRANSFORMARLOS CADA UNO DE UNA MANERA

In [10]:
df_competencia = df[df["competencia"] == True] 

In [11]:
df_propios = df[df["competencia"] == False] 

In [12]:
df_propios["id_hotel"] = df_propios.groupby("nombre_hotel").ngroup()+1

C:\Users\victo\AppData\Local\Temp\ipykernel_25132\3117162419.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_propios["id_hotel"] = df_propios.groupby("nombre_hotel").ngroup()+1


In [13]:
df_propios["estrellas"] = round(df_propios.groupby("id_hotel")["estrellas"].transform("mean"),2)

C:\Users\victo\AppData\Local\Temp\ipykernel_25132\1608655818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_propios["estrellas"] = round(df_propios.groupby("id_hotel")["estrellas"].transform("mean"),2)


**PARA EVITAR RUIDO VAMOS A CAMBIAR EL NOMBRE DE LA COLUMNA A `valoracion`**

In [14]:
df_propios.rename(columns={"estrellas": "valoracion"}, inplace=True)
df_competencia.rename(columns={"estrellas": "valoracion"}, inplace=True)

C:\Users\victo\AppData\Local\Temp\ipykernel_25132\3208228328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_propios.rename(columns={"estrellas": "valoracion"}, inplace=True)
C:\Users\victo\AppData\Local\Temp\ipykernel_25132\3208228328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_competencia.rename(columns={"estrellas": "valoracion"}, inplace=True)


In [15]:
df["ciudad"] = df["ciudad"].replace("", "Madrid")

# 2. NULOS

In [16]:
# Eliminar los valores nulos (NaT) y obtener el primer (y único valor)
fecha_inicio = df_propios["inicio_estancia"].dropna().unique()[0]
df_propios["inicio_estancia"] = df_propios["inicio_estancia"].fillna(fecha_inicio)

fecha_fin = df_propios["final_estancia"].dropna().unique()[0]
df_propios["final_estancia"] = df_propios["final_estancia"].fillna(fecha_fin)


# Rellenar los valores nulos con la media del precio por cada hotel
df_propios["precio_noche"] = df_propios["precio_noche"].fillna(
    df_propios.groupby("nombre_hotel")["precio_noche"].transform("mean")
    )


C:\Users\victo\AppData\Local\Temp\ipykernel_25132\3908642250.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_propios["inicio_estancia"] = df_propios["inicio_estancia"].fillna(fecha_inicio)
C:\Users\victo\AppData\Local\Temp\ipykernel_25132\3908642250.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_propios["final_estancia"] = df_propios["final_estancia"].fillna(fecha_fin)
C:\Users\victo\AppData\Local\Temp\ipykernel_25132\3908642250.py:10: SettingWithCopyWarning: 
A value is trying to be set on a

In [17]:
# Eliminar los valores nulos (NaT) y obtener el primer (y único valor)
fecha_inicio = df_competencia["inicio_estancia"].dropna().unique()[0]
df_competencia["inicio_estancia"] = df_competencia["inicio_estancia"].fillna(fecha_inicio)

fecha_fin = df_competencia["final_estancia"].dropna().unique()[0]
df_competencia["final_estancia"] = df_competencia["final_estancia"].fillna(fecha_fin)

C:\Users\victo\AppData\Local\Temp\ipykernel_25132\2561489396.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_competencia["inicio_estancia"] = df_competencia["inicio_estancia"].fillna(fecha_inicio)
C:\Users\victo\AppData\Local\Temp\ipykernel_25132\2561489396.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_competencia["final_estancia"] = df_competencia["final_estancia"].fillna(fecha_fin)


### GUARDAMOS LOS DATAFRAMES

In [18]:
df_propios.to_pickle("../data_2/limpieza/hoteles_propios_limpios.pkl")

In [19]:
df_competencia.to_pickle("../data_2/limpieza/hoteles_competencia.pkl")